import libraries

In [1]:
import tensorflow as tf
from keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.merge import concatenate
from keras import optimizers
import keras.backend as K
import scipy.stats as ss
import time
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

/Users/hanglou/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Black-Scholes formulas

In [ ]:
#Black and Scholes
def d1(St, Sk, r, sigma, T,t):
    return (np.log(St/Sk) + (r + sigma**2 / 2) * (T-t))/(sigma * np.sqrt(T-t))
 
def d2(St, Sk, r, sigma,T):
    return (np.log(St / Sk) + (r - sigma**2 / 2) * T) / (sigma * np.sqrt(T))
 
def BlackScholes(type,St, Sk, r, sigma, T,t):
    if type=="C":
        return St * ss.norm.cdf(d1(St, Sk, r, sigma, T,t)) - Sk * np.exp(-r * T) * ss.norm.cdf(d2(St, Sk, r, sigma, T))
    else:
       return Sk * np.exp(-r * T) * ss.norm.cdf(-d2(St, Sk, r, sigma, T)) - St * ss.norm.cdf(-d1(St, Sk, r, sigma, T,t))


# Model 1: fixed sigma in the Black-Scholes model


Data simulations for Model 1, fixed sigma in the Black-Scholes model

In [ ]:
n=100
W_incre=np.random.normal(0,1/10,(int(1e5),n))
W=np.concatenate((np.zeros((int(1e5),1)),np.cumsum(W_incre,axis=1)),axis=1)
t=np.linspace(0,1,100+1)
mu,sigma=0,0.1
S0=10
S=S0*np.exp((mu-1/2*sigma**2)*t+sigma*W)

In [ ]:
fig = pylab.figure()    
ax = fig.add_subplot(1,1,1)
ax.set_axisbelow(True)
ax.yaxis.grid(color='gray', linestyle='dashed')
plt.rcParams['figure.figsize']="20,7"
plt.rcParams.update({'font.size': 20})
plt.plot(t,S[0,:],t,S[1,:],t,S[2,:])
line1,=plt.plot(t,np.zeros(101)+10,linestyle='dotted',label='Strike price',color='red')
first_legend = plt.legend(handles=[line1], loc=1)
plt.title('Hedging European call option ')
plt.ylabel('Stock price')

In [ ]:
fig = pylab.figure()    
ax = fig.add_subplot(1,1,1)
ax.set_axisbelow(True)
ax.yaxis.grid(color='gray', linestyle='dashed')
plt.rcParams['figure.figsize']="20,7"
plt.rcParams.update({'font.size': 20})
plt.plot(t,S[0,:],t,S[1,:],t,S[2,:])
line1,=plt.plot(t,np.zeros(101)+10,linestyle='dotted',label='Strike price',color='red')
first_legend = plt.legend(handles=[line1], loc=1)
plt.title('Hedging European call option ')
plt.ylabel('Stock price')

Black-Schole delta

In [ ]:
Delta=np.zeros((100000,100))
for i in range(100000):
    for j in range(100):
        Delta[i,j]=ss.norm.cdf(d1(S[i,j],8,0,0.1,1,t[j]))
        

BS call option price

In [ ]:
V_0=BlackScholes("C",10,10,0,0.1,1,0)
V_0

loss function

In [ ]:
def Customloss(trueval,predictions):
    loss= K.square(V_0+K.sum(predictions*trueval,axis=-1)-K.maximum(S0+K.sum(trueval,axis=-1)-8,0.))
    return loss

Build model with keras API

In [ ]:
inputlist=[None]*100
outputlist=[None]*100
layer1=Dense(100,activation='relu')
layer2=Dense(100,activation='relu')
layer3=Dense(100,activation='relu')
layer4=Dense(1,activation='sigmoid')
for i in range(100):
    inputs= Input(shape=(2,))
    hidden1=layer1(inputs)
    hidden2=layer2(hidden1)
    hidden3=layer3(hidden2)
    outputlist[i]=layer4(hidden3)
    inputlist[i]=inputs
predictions=concatenate(outputlist)
model=Model(inputs=inputlist,outputs=predictions)
print(model.summary())

Input data to the model

In [ ]:
X=[None]*100

Y=S[:,1:101]-S[:,0:100]
for i in range(100):
    X[i]=np.concatenate((np.full((int(1e5),1),t[i]),S[:,i].reshape(int(1e5),1)), axis=1)


compile model with adam algorithm

In [ ]:
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss=Customloss,optimizer='adam')

model fitting

In [ ]:
model.fit(X,Y,validation_split=0.1,epochs=200,batch_size=100,verbose=1)

model analysis with plots

In [ ]:
def hedgeerror(trueval,predictions):
    loss=np.zeros(10000)
    for i in range(10000):
        loss[i]=V_0+np.sum(predictions[i,:]*trueval[i,:],axis=-1)-np.maximum(S0+np.sum(trueval[i,:],axis=-1)-8,0.)
    return loss

In [ ]:
hedge_error=hedgeerror(Y[90000:100000,:],prediction[90000:100000,:])

In [ ]:
Delta_error=hedgeerror(Y[90000:100000,:],Delta[90000:100000,:])

In [ ]:
n, bins, patches = plt.hist(hedge_error,80, density=True,color='green', alpha=0.6,label='Model hedging error ')
n, bins, patches = plt.hist(Delta_error,80, density=True,color='red', alpha=0.6,label='BS hedging error')
plt.xlabel('Hedging error')
plt.ylabel('Density')
plt.title('Histogram of hedging error')
plt.legend()
plt.grid(True)

In [ ]:
Stest=np.linspace(7,14,71)
D=np.zeros((71,100))
for i in range(71):
    for j in range(100):
        D[i,j]=ss.norm.cdf(d1(Stest[i],8,0,0.1,1,t[j]))
Xt=[None]*71
for j in range(71):
    Xtest=[None]*100
    for i in range(100):
        Xtest[i]=np.array([[t[i],Stest[j]]])
    Xt[j]=Xtest
Stock,time=np.meshgrid(Stest,t[0:100])

In [ ]:
plt.rcParams['figure.figsize']="15,7"
plt.rcParams.update({'font.size': 20})
plt.rcParams['axes.labelpad'] = 20
figd = plt.figure()
ax = Axes3D(figd)
ax.plot_surface(Stock, time,np.transpose(d), rstride=1, cstride=1, cmap=cm.coolwarm, shade='interp')
plt.title('Model Delta')
ax.view_init(27, -125)


ax.set_xlabel('Stock Price')
ax.set_ylabel('Time to Maturity')
ax.set_zlabel('Delta')

# Model 2: unknown sigma in the Black-Schole model 

data simulation

In [ ]:
Sigma=0.3+0.1*np.random.rand(100000,1)
S_current=np.zeros((100000,101))
for i in range(100000):
    S_current[i,:]=  S0*np.exp((mu-1/2*Sigma[i,:]**2)*t+Sigma[i,:]*W[i,:])  

In [ ]:
W_incre1=np.random.normal(0,(1/1000)**0.5,(int(1e5),300))
W1=np.concatenate((np.zeros((int(1e5),1)),np.cumsum(W_incre1,axis=1)),axis=1)
S_hist=np.zeros((100000,301))
for i in range(100000):
    S_hist[i,:]=  S0*np.exp((mu-1/2*Sigma[i,:]**2)*t1+Sigma[i,:]*W1[i,:])  
R_hist=np.zeros((100000,300))
for i in range(10000):
    R_hist[i,:]=(S_hist[i,1:301]-S_hist[i,0:300])/S_hist[i,0:300]

In [ ]:
plt.rcParams.update({'font.size': 14})
plt.plot(t,S_current[0,:],t,S_current[1,:],t,S_current[2,:],t,S_current[3,:],t,S_current[4,:])
plt.title('5 Simulated stock paths ')
plt.xlabel('time')
plt.ylabel('price')


In [ ]:
plt.plot(t1,S_hist[0,:],t1,S_hist[1,:],t1,S_hist[2,:],t1,S_hist[3,:],t1,S_hist[4,:])
plt.title('5 Simulated stock paths ')
plt.xlabel('time')
plt.ylabel('price')

Black-Scholes delta

In [ ]:
Delta1=np.zeros((100000,100))
for i in range(100000):
    for j in range(100):
        Delta1[i,j]=ss.norm.cdf(d1(S_current[i,j],12,0,Sigma[i,0],1,t[j]))

call option prices

In [ ]:
V0arr=np.zeros((100000,1))

for i in range(100000):
     V0arr[i,:]=BlackScholes("C",10,10,0,Sigma[i,0],1,0)

The model which uses Sigma as a input directly

layer specification

In [ ]:
inputlist=[None]*102
outputlist=[None]*100

input_V0=Input(shape=(1,),name='input_V0')

input_sigma=Input(shape=(1,),name='input_sigma')

layer1=Dense(100,activation='relu')
layer2=Dense(100,activation='relu')
layer3=Dense(1,activation='sigmoid')
inputlist[100]=input_V0
inputlist[101]=input_sigma

for i in range(100):
    inputs=Input(shape=(2,))
    merge=keras.layers.concatenate([inputs, input_sigma])

    hidden6=layer1(merge)
    hidden7=layer2(hidden6)
    
    outputlist[i]=layer3(hidden7)
    
    inputlist[i]=inputs
predictions=concatenate(outputlist)
model3=Model(inputs=inputlist,outputs=predictions)

Loss function

In [ ]:
def newloss(V_0):
    def Customloss(y_true,y_pred):
        loss= K.square(V_0+K.sum(y_pred*y_true,axis=-1)-K.maximum(S0+K.sum(y_true,axis=-1)-10,0.))
        return loss
    return Customloss

Input data

In [ ]:
X=[None]*102

Y=S_current[:,1:101]-S_current[:,0:100]
for i in range(100):
    X[i]=np.concatenate((np.full((100000,1),t[i]),S_current[:,i].reshape(100000,1)), axis=1)
X[100]=V0arr
X[101]=Sigma

In [ ]:
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model3.compile(loss=newloss(input_V0),optimizer='adam')

In [ ]:
model3.fit(X,Y,validation_split=0.1,epochs=30,batch_size=100,verbose=1)

In [ ]:
Portfolio3=model3.predict(X)

Model uses historical data

In [ ]:
inputlist=[None]*102
outputlist=[None]*100

input_V0=Input(shape=(1,),name='input_V0')
input_hist=Input(shape=(300,),name='R_hist')

hidden1=Dense(2000,activation='relu')(input_hist)
hidden2=Dense(1000,activation='relu')(hidden1)
hidden3=Dense(200,activation='relu')(hidden2)

sigma=Dense(1,activation='sigmoid')(hidden3)

layer1=Dense(100,activation='relu')
layer2=Dense(100,activation='relu')
layer3=Dense(1,activation='sigmoid')
inputlist[100]=input_V0
inputlist[101]=input_hist

for i in range(100):
    inputs=Input(shape=(2,))
    merge=keras.layers.concatenate([inputs, sigma])

    hidden6=layer1(merge)
    hidden7=layer2(hidden6)
    
    outputlist[i]=layer3(hidden7)
    
    inputlist[i]=inputs
predictions=concatenate(outputlist)
model2=Model(inputs=inputlist,outputs=predictions)

In [ ]:
def newloss(V_0):
    def Customloss(y_true,y_pred):
        loss= K.square(V_0+K.sum(y_pred*y_true,axis=-1)-K.maximum(S0+K.sum(y_true,axis=-1)-10,0.))
        return loss
    return Customloss

In [ ]:
X=[None]*102

Y=S_current[:,1:101]-S_current[:,0:100]
for i in range(100):
    X[i]=np.concatenate((np.full((100000,1),t[i]),S_current[:,i].reshape(100000,1)), axis=1)
X[100]=V0arr 
X[101]=R_hist


In [ ]:
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model2.compile(loss=newloss(input_V0),optimizer='adam')

In [ ]:
model2.fit(X,Y,validation_split=0.1,epochs=30,batch_size=100,verbose=1)

In [ ]:
Portfolio2=model2.predict(X)

Some plots for analysing data

In [ ]:
plt.plot(t[0:100],Portfolio2[55296,:],label='Model Delta')
plt.plot(t[0:100],Delta1[55296,:],label='BS Delta')
plt.legend(loc='lower right')
plt.title('BS delta against Neural network delta of a sampel trajectory with sigma=0.40')
plt.xlabel('time')
plt.ylabel('hedging ratio')
plt.legend()

In [ ]:
S05=S_current[9000:10000,50]
Delta05=Delta1[9000:10000,50]
Portfolio05=Portfolio2[9000:10000,50]
Delsig=Portfolio3[9000:10000,50]

In [ ]:
plt.rcParams['figure.figsize']="13,8"
plt.rcParams.update({'font.size': 15})
plt.rcParams['axes.labelpad'] = 25
figd = plt.figure()
ax = figd.add_subplot(111, projection='3d')
ax.scatter(S05, Sigma[9000:10000,0],Portfolio05 , c='r',label='model delta')
ax.scatter(S05, Sigma[9000:10000,0],Delta05 , c='b',label='BS delta')
ax.set_xlabel('Stock Price')
ax.set_ylabel('Sigma')
ax.set_zlabel('Delta')
plt.legend()
plt.title('Model Delta vs BS Delta')
plt.show()

In [ ]:
def hedgeerror(trueval,predictions,arr):
    loss=np.zeros(10000)
    for i in range(10000):
        loss[i]=arr[i,:]+np.sum(predictions[i,:]*trueval[i,:],axis=-1)-np.maximum(S0+np.sum(trueval[i,:],axis=-1)-12,0.)
    return loss

In [ ]:
n, bins, patches = plt.hist(model_error,80, density=True,color='green', alpha=0.6,label='Model hedging error(using historical data)   ')
n, bins, patches = plt.hist(Delta_error,80, density=True,color='red', alpha=0.6,label='BS hedging error')
plt.xlabel('Hedging error')
plt.ylabel('Density')
plt.title('Histogram of hedging error')
plt.legend()
plt.grid(True)

# Model 3: fixed sigma in an incomplete model

data simulations

In [ ]:
S0=10
strike=10
mu1=0
sigma1=0.3
S1=S0*np.exp((mu1-1/2*sigma1**2)*t+sigma1*W)

In [ ]:
plt.plot(t,S1[0,:],t,S1[1,:],t,S1[2,:],t,S1[3,:],t,S1[4,:])
plt.title('5 Simulated stock paths ')
plt.xlabel('time')
plt.ylabel('price')

define loss funtion with exponential utility function, no transaction cost

In [ ]:
lam=0.1
def Utilityloss1(y_true,y_pred):
        loss= K.exp(-lam*(K.sum(y_pred*y_true,axis=-1)-K.maximum(S0+K.sum(y_true,axis=-1)-strike,0.)))
        return loss
def Utilityloss2(y_true,y_pred):
    loss=K.exp(-lam*K.sum(y_pred*y_true,axis=-1))
    return loss

In [ ]:
inputlist=[None]*100
outputlist=[None]*100
layer1=Dense(100,activation='relu')
layer2=Dense(100,activation='relu')
layer3=Dense(100,activation='relu')
layer4=Dense(1,activation='sigmoid')
for i in range(100):
    inputs= Input(shape=(2,))
    hidden1=layer1(inputs)
    hidden2=layer2(hidden1)
    hidden3=layer3(hidden2)
    outputlist[i]=layer4(hidden3)
    inputlist[i]=inputs
predictions=concatenate(outputlist)
model4=Model(inputs=inputlist,outputs=predictions)

In [ ]:
inputlist=[None]*100
outputlist=[None]*100
layer1=Dense(100,activation='relu')
layer2=Dense(100,activation='relu')
layer3=Dense(100,activation='relu')
layer4=Dense(1,activation='sigmoid')

for i in range(100):
    inputs= Input(shape=(2,))
    hidden1=layer1(inputs)
    hidden2=layer2(hidden1)
    hidden3=layer3(hidden2)
    outputlist[i]=layer4(hidden3)
    inputlist[i]=inputs
predictions=concatenate(outputlist)
model5=Model(inputs=inputlist,outputs=predictions)

In [ ]:
X=[None]*100

Y=S1[:,1:101]-S1[:,0:100]
for i in range(100):
    X[i]=np.concatenate((np.full((int(1e5),1),t[i]),S1[:,i].reshape(int(1e5),1)), axis=1)

In [ ]:
adam = optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model4.compile(loss=Utilityloss1,optimizer='adam')
model5.compile(loss=Utilityloss2,optimizer='adam')

In [ ]:
model4.fit(X,Y,validation_split=0.1,epochs=20,batch_size=100,verbose=1)

In [ ]:
model5.fit(X,Y,epochs=3,batch_size=1000,verbose=1)

In [ ]:
Portfolio05=model2.predict(X)

indifference pricing

In [ ]:
P_0=1/lam*np.log(model2.evaluate(X,Y))

some plots for analysing

In [ ]:
def hedgeerror(trueval,predictions):
    loss=np.zeros(10000)
    for i in range(10000):
        loss[i]=P_0+np.sum(predictions[i,:]*trueval[i,:],axis=-1)-np.maximum(S0+np.sum(trueval[i,:],axis=-1)-10,0.)
    return loss

In [ ]:
exp_pl3=hedgeerror(Y[90000:100000,:],Portfolio3[90000:100000,:])

In [ ]:
plt.rcParams['figure.figsize']="15,7"
plt.rcParams.update({'font.size': 20})
plt.rcParams['axes.labelpad'] = 20
n, bins, patches = plt.hist(exp_pl01,80, density=True,color='green', alpha=0.4,label=' $\lambda=0.01$')
n, bins, patches = plt.hist(exp_pl05,80, density=True,color='red', alpha=0.4,label='$\lambda=0.5$')
n, bins, patches = plt.hist(exp_pl1,80, density=True,color='blue', alpha=0.4,label='$\lambda=1$')
plt.xlabel('Profit & LOss')
plt.ylabel('Density')
plt.title('Profit & Loss for different risk-aversion parameter ')
plt.legend()
plt.grid(True)

In [ ]:
d1=np.zeros((71,100))
for i in range(71):
    d1[i,:]=model2.predict(Xt[i])

In [ ]:
plt.rcParams['figure.figsize']="15,7"
plt.rcParams.update({'font.size': 20})
plt.rcParams['axes.labelpad'] = 20
figd = plt.figure()
ax = Axes3D(figd)
ax.plot_surface(Stock, time,np.transpose(d05), rstride=1, cstride=1, cmap=cm.coolwarm, shade='interp')
plt.title(' Hedgeing Strategy for $\lambda=0.5$')
ax.view_init(27, -125)


ax.set_xlabel('Stock Price')
ax.set_ylabel('Time')
ax.set_zlabel('Hedging ratio')

Model with proportional transaction costs

In [ ]:
lam=0.5
c=0.02
def loss_cost1(stock):
    def Utilityloss3(y_true,y_pred):
        y_incre=y_pred-K.concatenate([K.zeros(shape=(100,1)),y_pred[:,:100]],axis=-1)
        loss= K.exp(-lam*(K.sum(y_pred[:,:100]*y_true,axis=-1)-K.maximum(S0+K.sum(y_true,axis=-1)-strike,0.)-c*(K.sum(K.abs(y_incre)*stock,axis=-1))))
        return loss
    return Utilityloss3
def loss_cost2(stock):                   
    def Utilityloss4(y_true,y_pred):
        y_incre=y_pred-K.concatenate([K.zeros(shape=(100,1)),y_pred[:,:100]],axis=-1)
        loss=K.exp(-lam*(K.sum(y_pred[:,:100]*y_true,axis=-1)-c*(K.sum(K.abs(y_incre)*stock,axis=-1))))
        return loss
    return Utilityloss4

In [ ]:
inputlist1=[None]*102
outputlist1=[None]*101
input_S1=Input(shape=(101,),name='input_S1')

layer1=Dense(100,activation='relu',name='layer1')
layer2=Dense(100,activation='relu',name='layer2')
layer3=Dense(100,activation='relu',name='layer3')
layer4=Dense(1,activation='sigmoid',name='layer4')

inputlist1[101]=input_S1
for i in range(101):
    inputs= Input(shape=(2,))
    hidden1=layer1(inputs)
    hidden2=layer2(hidden1)
    hidden3=layer3(hidden2)
    outputlist1[i]=layer4(hidden3)
    inputlist1[i]=inputs
predictions=concatenate(outputlist1)
model_cost1=Model(inputs=inputlist1,outputs=predictions)

In [ ]:
inputlist2=[None]*102
outputlist2=[None]*101
input_S2=Input(shape=(101,),name='input_S2')
layer1=Dense(100,activation='relu',name='layer1')
layer2=Dense(100,activation='relu',name='layer2')
layer3=Dense(100,activation='relu',name='layer3')
layer4=Dense(1,activation='sigmoid',name='layer4')
inputlist2[101]=input_S2
for i in range(101):
    inputs= Input(shape=(2,))
    hidden1=layer1(inputs)
    hidden2=layer2(hidden1)
    hidden3=layer3(hidden2)
    outputlist2[i]=layer4(hidden3)
    inputlist2[i]=inputs
predictions=concatenate(outputlist2)
model_cost2=Model(inputs=inputlist2,outputs=predictions)

In [ ]:
X=[None]*102

Y=S1[:,1:101]-S1[:,0:100]
for i in range(101):
    X[i]=np.concatenate((np.full((int(1e5),1),t[i]),S1[:,i].reshape(int(1e5),1)), axis=1)
X[101]=S1[:,0:101]

In [ ]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model_cost1.compile(loss=loss_cost1(input_S1),optimizer='adam')

In [ ]:
model_cost1.fit(X,Y,validation_split=0.1,epochs=50,batch_size=100,verbose=1)

In [ ]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model_cost2.compile(loss=loss_cost2(input_S2),optimizer='adam')


In [ ]:
model_cost2.fit(X,Y,epochs=2,batch_size=100,verbose=1)

indifference pricing

In [ ]:
P_0=1/lam*np.log(model_cost1.evaluate(X,Y,batch_size=100))

In [ ]:
Portfolio1_cost05=model_cost1.predict(X)

plots

In [ ]:
Xt=[None]*71
for j in range(71):
    Xtest=[None]*102
    for i in range(101):
        Xtest[i]=np.array([[t[i],Stest[j]]])
    Xtest[101]=S1    
    Xt[j]=Xtest

In [ ]:
d1=np.zeros((71,101))
for i in range(71):
    d1[i,:]=model_cost1.predict(Xt[i])

In [ ]:
Stock,time=np.meshgrid(Stest,t[0:101])

In [ ]:
plt.rcParams['figure.figsize']="15,7"
plt.rcParams.update({'font.size': 20})
plt.rcParams['axes.labelpad'] = 20
figd = plt.figure()
ax = Axes3D(figd)
ax.plot_surface(Stock, time,np.transpose(d1), rstride=1, cstride=1, cmap=cm.coolwarm, shade='interp')
plt.title(' Hedgeing Strategy for $\lambda=1$ with transaction cost')
ax.view_init(27, -125)

ax.set_zlim([0,1])
ax.set_xlabel('Stock Price')
ax.set_ylabel('Time ')
ax.set_zlabel('Hedging ratio')

In [ ]:
def hedgeerror(trueval,predictions,stock):
    loss=np.zeros(100000)
    for i in range(100000):
        y_incre=predictions[i,:]-np.concatenate([np.zeros(1),predictions[i,:100]],axis=-1)
        loss[i]=p1_05+np.sum(predictions[i,:100]*trueval[i,:],axis=-1)-np.maximum(S0+np.sum(trueval[i,:],axis=-1)-10,0.)-c*(np.sum(np.abs(y_incre)*stock[i,:],axis=-1))
    return loss

In [ ]:
pl1_cost05=hedgeerror(Y,Portfolio1_cost05,S1)

In [ ]:
plt.rcParams['figure.figsize']="15,7"
plt.rcParams.update({'font.size': 20})
plt.rcParams['axes.labelpad'] = 20
n, bins, patches = plt.hist(pl1_cost01,80, density=True,color='green', alpha=0.4,label=' $\lambda=0.1$')
n, bins, patches = plt.hist(pl1_cost05,80, density=True,color='red', alpha=0.4,label='$\lambda=0.5$')
n, bins, patches = plt.hist(pl1_cost1,80, density=True,color='blue', alpha=0.4,label='$\lambda=1$')
plt.xlabel('Profit & LOss')
plt.ylabel('Density')
plt.title('Profit & Loss for different risk-aversion parameter with transaction cost')
plt.legend()
plt.grid(True)